In [13]:
# Initial imports
import os
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools_fp import MCSimulation
import datetime
import logging
import csv
import json

%matplotlib inline

In [14]:
# #Load .env enviroment variables
# load_dotenv()

In [16]:
bpip_delta=datetime.timedelta(weeks=1)
bpip_delta10=datetime.timedelta(weeks=12)
bpip_delta30=datetime.timedelta(weeks=205)

# ISO 8601 week as number with Monday as the first day of the week then change date from string to int
bpip_date = date.today()
bpip_date_cast = bpip_date + bpip_delta10
bpip_date_hist = bpip_date - bpip_delta30
bpip_day_fdiff = (bpip_date_cast - bpip_date) / 7
bpip_day_hdiff = (bpip_date - bpip_date_hist) / 7
bpip_fdiff = bpip_day_fdiff.days
bpip_hdiff = bpip_day_hdiff.days


In [18]:
#Loop through url calendars for forcast data
count=0
df_ffxevents_list = []
while(count < bpip_fdiff):
    bpip_dfdelta=datetime.timedelta(weeks=count)
    bpip_date_dfcast = bpip_date + bpip_dfdelta
    print(f"Forecast Date: {bpip_date_dfcast}")
    bpip_wk_dfcaststr = bpip_date_dfcast.strftime("%V")
    print(f"Forecast Week String: {bpip_wk_dfcaststr}")
    bpip_yr_dfcaststr = bpip_date_dfcast.strftime("%Y")
    print(f"Current Year String: {bpip_yr_dfcaststr}")
    bpip_yr_dfcastiso = bpip_date_dfcast.isocalendar()[0]
    print(f"Current Year: {bpip_yr_dfcastiso}")
    # get webpage and make the soup
    babe_url = f"https://www.babypips.com/economic-calendar?timezone=America%2FChicago&week={bpip_yr_dfcastiso}-W{bpip_wk_dfcaststr}"
    print(babe_url)
    baseURL = requests.get(babe_url)
    fxcal_soup = BeautifulSoup(baseURL.content, 'html.parser')

    #Get all tag decendents 
    fxcal = fxcal_soup.find(class_="Table-module__table___2Nidf")
    fxcal_tbl_days = fxcal.find_all(class_="Section-module__container___2mwAW")

    #Loop through html classes and div tag elements
    #df_ffxevents_list = []
    for x in fxcal_tbl_days[0:]:
        pd_news_day = [a.get_text() for a in x.select(".Section-module__container___2mwAW .Table-module__name___1XC4z")]
        pd_month_day = [b.get_text() for b in x.select(".Section-module__container___2mwAW .Table-module__month___2Efxb") * len(pd_news_day)]
        pd_date_day = [c.get_text() for c in x.select(".Section-module__container___2mwAW .Table-module__dayNumber___1DiwN") * len(pd_news_day)]
        pd_wkday_day = [d.get_text() for d in x.select(".Section-module__container___2mwAW .Table-module__weekday___1xUx_") * len(pd_news_day)]
        pd_time_day = [e.get_text() for e in x.select(".Section-module__container___2mwAW .Table-module__time___iPjt3")]
        pd_curr_day = [f.get_text() for f in x.select(".Section-module__container___2mwAW .Currency-module__code___1A32e")]
        pd_impact_day = [g.get_text() for g in x.select(".Section-module__container___2mwAW .Impact-module__pill___2pcuA")]
        pd_forecast_day = [h.get_text() for h in x.select(".Table-module__eventRow___2Z1BD .Table-module__forecast___3F_kh")]
        pd_previous_day = [i.get_text() for i in x.select(".Table-module__eventRow___2Z1BD .Table-module__previous___1PJZ9")]

    #Build list elements into list of dataframes, concatenate dataframes and export to csv 
        df_ffxevents_day = pd.DataFrame.from_dict({
        "pd_year": bpip_yr_dfcastiso,
        "pd_month": pd_month_day,
        "pd_date": pd_date_day,
        "pd_weekday": pd_wkday_day,
        "pd_time": pd_time_day,
        "pd_currency": pd_curr_day,
        "pd_news": pd_news_day,
        "pd_impact": pd_impact_day,
        "pd_forecast": pd_forecast_day,
        "pd_previous": pd_previous_day})
        df_ffxevents_list.append(df_ffxevents_day)
    count = count + 1
df_ffxevents_cat = pd.concat(df_ffxevents_list)
df_ffxevents_cat.to_csv('fxevents_forecast.csv')
count=0
df_ffxevents_cat.head(20)


Forecast Date: 2020-12-27
Forecast Week String: 52
Current Year String: 2020
Current Year: 2020
Forecast Date: 2021-01-03
Forecast Week String: 53
Current Year String: 2021
Current Year: 2020
Forecast Date: 2021-01-10
Forecast Week String: 01
Current Year String: 2021
Current Year: 2021
Forecast Date: 2021-01-17
Forecast Week String: 02
Current Year String: 2021
Current Year: 2021
Forecast Date: 2021-01-24
Forecast Week String: 03
Current Year String: 2021
Current Year: 2021
Forecast Date: 2021-01-31
Forecast Week String: 04
Current Year String: 2021
Current Year: 2021
Forecast Date: 2021-02-07
Forecast Week String: 05
Current Year String: 2021
Current Year: 2021
Forecast Date: 2021-02-14
Forecast Week String: 06
Current Year String: 2021
Current Year: 2021
Forecast Date: 2021-02-21
Forecast Week String: 07
Current Year String: 2021
Current Year: 2021
Forecast Date: 2021-02-28
Forecast Week String: 08
Current Year String: 2021
Current Year: 2021
Forecast Date: 2021-03-07
Forecast Week 

,pd_year,pd_month,pd_date,pd_weekday,pd_time,pd_currency,pd_news,pd_impact,pd_forecast,pd_previous
0,2020,Dec,20,Sunday,19:30,CNY,Chinese Loan Prime Rate 1Y,high,3.85%,3.85%
1,2020,Dec,20,Sunday,19:30,CNY,Chinese Loan Prime Rate 5Y,med,4.65%,4.65%
2,2020,Dec,20,Sunday,20:00,NZD,Credit Card Spending YoY,low,,-6.0%*
0,2020,Dec,21,Monday,All Day,JPY,BoJ Gov Kuroda Speech,med,,
1,2020,Dec,21,Monday,05:00,GBP,CBI Distributive Trades,med,-4.0,-25.0
2,2020,Dec,21,Monday,07:30,USD,Chicago Fed National Activity Index,med,0.4,1.01*
3,2020,Dec,21,Monday,07:30,CAD,New Housing Price Index YoY,med,4.0%,3.9%
4,2020,Dec,21,Monday,07:30,CAD,New Housing Price Index MoM,med,0.7%,0.8%
5,2020,Dec,21,Monday,07:30,CAD,Wholesale Sales MoM Flash,low,,1.0%
6,2020,Dec,21,Monday,08:00,EUR,French 6-Month BTF Auction,low,,-0.72%*


In [19]:
file_path = Path("fxevents_forecast.csv")
forecast_data = pd.read_csv(file_path)
forecast_data.head(20)

,Unnamed: 0,pd_year,pd_month,pd_date,pd_weekday,pd_time,pd_currency,pd_news,pd_impact,pd_forecast,pd_previous
0,0,2020,Dec,20,Sunday,19:30,CNY,Chinese Loan Prime Rate 1Y,high,3.85%,3.85%
1,1,2020,Dec,20,Sunday,19:30,CNY,Chinese Loan Prime Rate 5Y,med,4.65%,4.65%
2,2,2020,Dec,20,Sunday,20:00,NZD,Credit Card Spending YoY,low,NaN,-6.0%*
3,0,2020,Dec,21,Monday,All Day,JPY,BoJ Gov Kuroda Speech,med,NaN,NaN
4,1,2020,Dec,21,Monday,05:00,GBP,CBI Distributive Trades,med,-4.0,-25.0
5,2,2020,Dec,21,Monday,07:30,USD,Chicago Fed National Activity Index,med,0.4,1.01*
6,3,2020,Dec,21,Monday,07:30,CAD,New Housing Price Index YoY,med,4.0%,3.9%
7,4,2020,Dec,21,Monday,07:30,CAD,New Housing Price Index MoM,med,0.7%,0.8%
8,5,2020,Dec,21,Monday,07:30,CAD,Wholesale Sales MoM Flash,low,NaN,1.0%
9,6,2020,Dec,21,Monday,08:00,EUR,French 6-Month BTF Auction,low,NaN,-0.72%*


In [20]:
forecast_data['Date'] = pd.to_datetime(forecast_data[['pd_month', 'pd_date', 'pd_year']].astype(str).agg('-'.join, axis=1))
forecast_data.set_index(forecast_data['Date'], inplace=True)
forecast_data = forecast_data.drop(columns=["pd_month", "pd_date", "pd_year", "pd_weekday", "Date"])
forecast_data = forecast_data.loc[:, ~forecast_data.columns.str.contains('^Unnamed')]
forecast_data.head(20)

,pd_time,pd_currency,pd_news,pd_impact,pd_forecast,pd_previous
Date,,,,,,
2020-12-20,19:30,CNY,Chinese Loan Prime Rate 1Y,high,3.85%,3.85%
2020-12-20,19:30,CNY,Chinese Loan Prime Rate 5Y,med,4.65%,4.65%
2020-12-20,20:00,NZD,Credit Card Spending YoY,low,NaN,-6.0%*
2020-12-21,All Day,JPY,BoJ Gov Kuroda Speech,med,NaN,NaN
2020-12-21,05:00,GBP,CBI Distributive Trades,med,-4.0,-25.0
2020-12-21,07:30,USD,Chicago Fed National Activity Index,med,0.4,1.01*
2020-12-21,07:30,CAD,New Housing Price Index YoY,med,4.0%,3.9%
2020-12-21,07:30,CAD,New Housing Price Index MoM,med,0.7%,0.8%
2020-12-21,07:30,CAD,Wholesale Sales MoM Flash,low,NaN,1.0%


In [21]:
forecast_data = forecast_data[forecast_data['pd_currency'].isin(['USD', 'EUR', 'GBP', 'JPY'])]
forecast_data = forecast_data[forecast_data['pd_impact'].isin(['high'])]
forecast_data.head(20)

,pd_time,pd_currency,pd_news,pd_impact,pd_forecast,pd_previous
Date,,,,,,
2020-12-22,01:00,EUR,German GfK Consumer Confidence,high,-9.2,-6.8*
2020-12-22,09:00,USD,CB Consumer Confidence,high,95.1,92.9*
2020-12-23,07:30,USD,Personal Income,high,-0.4%,-0.6%*
2020-12-23,07:30,USD,Personal Spending MoM,high,-0.4%,0.3%*
2020-12-23,07:30,USD,Durable Goods Orders,high,0.6%,1.8%*
2020-12-23,09:00,USD,UoM Consumer Sentiment Index,high,81.3,76.9
2021-01-05,23:00,JPY,Consumer Confidence,high,35.0,33.7
2021-01-06,07:15,USD,ADP National Employment Report,high,210.0k,307.0k
2021-01-07,07:30,USD,Balance of Trade,high,-66.0B,-63.1B


In [22]:
forecast_data.isnull().mean() * 100

pd_time         0.000000
pd_currency     0.000000
pd_news         0.000000
pd_impact       0.000000
pd_forecast    52.054795
pd_previous    58.904110
dtype: float64

In [23]:
forecast_data["pd_forecast"] = forecast_data["pd_forecast"].fillna("Unknown")
forecast_data["pd_previous"] = forecast_data["pd_previous"].fillna("Unknown")
forecast_data.head(20)

,pd_time,pd_currency,pd_news,pd_impact,pd_forecast,pd_previous
Date,,,,,,
2020-12-22,01:00,EUR,German GfK Consumer Confidence,high,-9.2,-6.8*
2020-12-22,09:00,USD,CB Consumer Confidence,high,95.1,92.9*
2020-12-23,07:30,USD,Personal Income,high,-0.4%,-0.6%*
2020-12-23,07:30,USD,Personal Spending MoM,high,-0.4%,0.3%*
2020-12-23,07:30,USD,Durable Goods Orders,high,0.6%,1.8%*
2020-12-23,09:00,USD,UoM Consumer Sentiment Index,high,81.3,76.9
2021-01-05,23:00,JPY,Consumer Confidence,high,35.0,33.7
2021-01-06,07:15,USD,ADP National Employment Report,high,210.0k,307.0k
2021-01-07,07:30,USD,Balance of Trade,high,-66.0B,-63.1B


In [24]:
forecast_data.isnull().mean() * 100

pd_time        0.0
pd_currency    0.0
pd_news        0.0
pd_impact      0.0
pd_forecast    0.0
pd_previous    0.0
dtype: float64

In [25]:
forecast_data = forecast_data.rename(columns={
    "pd_time": "Time",
    "pd_currency": "Currency",
    "pd_news": "News", 
    "pd_impact": "Impact",
    "pd_forecast": "Forecast",
    "pd_previous": "Previous"})
forecast_data.to_csv("Data/fxevents_forecast_GSd.csv")
forecast_data.head(20)

,Time,Currency,News,Impact,Forecast,Previous
Date,,,,,,
2020-12-22,01:00,EUR,German GfK Consumer Confidence,high,-9.2,-6.8*
2020-12-22,09:00,USD,CB Consumer Confidence,high,95.1,92.9*
2020-12-23,07:30,USD,Personal Income,high,-0.4%,-0.6%*
2020-12-23,07:30,USD,Personal Spending MoM,high,-0.4%,0.3%*
2020-12-23,07:30,USD,Durable Goods Orders,high,0.6%,1.8%*
2020-12-23,09:00,USD,UoM Consumer Sentiment Index,high,81.3,76.9
2021-01-05,23:00,JPY,Consumer Confidence,high,35.0,33.7
2021-01-06,07:15,USD,ADP National Employment Report,high,210.0k,307.0k
2021-01-07,07:30,USD,Balance of Trade,high,-66.0B,-63.1B


In [27]:
forecast_path = Path("Data/fxevents_forecast_master.csv")
forecastmstr_data = pd.read_csv(forecast_path, index_col="Date", infer_datetime_format=True, parse_dates=True)
forecastmstr_data.head(20)

,Time,Currency,News,Impact,Forecast,Previous,Direction
Date,,,,,,,
2020-12-22,1:00,EUR,German GfK Consumer Confidence,high,-9.2,-6.8*,1
2020-12-22,9:00,USD,CB Consumer Confidence,high,95.1,92.9*,1
2020-12-23,7:30,USD,Personal Income,high,-0.40%,-0.6%*,1
2020-12-23,7:30,USD,Personal Spending MoM,high,-0.40%,0.3%*,1
2020-12-23,7:30,USD,Durable Goods Orders,high,0.60%,1.8%*,1
2020-12-23,9:00,USD,UoM Consumer Sentiment Index,high,81.3,76.9,1
2021-01-05,23:00,JPY,Consumer Confidence,high,35,33.7,1
2021-01-06,7:15,USD,ADP National Employment Report,high,210.0k,307.0k,1
2021-01-07,7:30,USD,Balance of Trade,high,-66.0B,-63.1B,1


In [28]:
#Loop through url calendars for historical data
count=bpip_hdiff
df_hfxevents_list = []
while(count > 0):
    bpip_dfdelta=datetime.timedelta(weeks=count)
    bpip_date_dfhist = bpip_date - bpip_dfdelta
    print(f"Forecast Date: {bpip_date_dfhist}")
    bpip_wk_dfhiststr = bpip_date_dfhist.strftime("%V")
    print(f"Forcast Week String: {bpip_wk_dfhiststr}")
    bpip_yr_dfhiststr = bpip_date_dfhist.strftime("%Y")
    print(f"Current Year String: {bpip_yr_dfhiststr}")
    bpip_yr_dfhistiso = bpip_date_dfhist.isocalendar()[0]
    print(f"Current Year: {bpip_yr_dfhistiso}")
    # get webpage and make the soup
    babe_url = f"https://www.babypips.com/economic-calendar?timezone=America%2FChicago&week={bpip_yr_dfhistiso}-W{bpip_wk_dfhiststr}"
    print(babe_url)
    baseURL = requests.get(babe_url)
    fxcal_soup = BeautifulSoup(baseURL.content, 'html.parser')

    #Get all tag decendents 
    fxcal = fxcal_soup.find(class_="Table-module__table___2Nidf")
    fxcal_tbl_days = fxcal.find_all(class_="Section-module__container___2mwAW")

    #Loop through html classes and div tag elements
    for y in fxcal_tbl_days[0:]:
        pd_news_day = [j.get_text() for j in y.select(".Section-module__container___2mwAW .Table-module__name___1XC4z")]
        pd_month_day = [k.get_text() for k in y.select(".Section-module__container___2mwAW .Table-module__month___2Efxb") * len(pd_news_day)]
        pd_date_day = [l.get_text() for l in y.select(".Section-module__container___2mwAW .Table-module__dayNumber___1DiwN") * len(pd_news_day)]
        pd_wkday_day = [m.get_text() for m in y.select(".Section-module__container___2mwAW .Table-module__weekday___1xUx_") * len(pd_news_day)]
        pd_time_day = [n.get_text() for n in y.select(".Section-module__container___2mwAW .Table-module__time___iPjt3")]
        pd_curr_day = [o.get_text() for o in y.select(".Section-module__container___2mwAW .Currency-module__code___1A32e")]
        pd_impact_day = [p.get_text() for p in y.select(".Section-module__container___2mwAW .Impact-module__pill___2pcuA")]
        pd_forecast_day = [q.get_text() for q in y.select(".Table-module__eventRow___2Z1BD .Table-module__forecast___3F_kh")]
        pd_previous_day = [r.get_text() for r in y.select(".Table-module__eventRow___2Z1BD .Table-module__previous___1PJZ9")]

    #Build list elements into list of dataframes, concatenate dataframes and export to csv 
        df_hfxevents_day = pd.DataFrame.from_dict({
        "pd_year": bpip_yr_dfhistiso,
        "pd_month": pd_month_day,
        "pd_date": pd_date_day,
        "pd_weekday": pd_wkday_day,
        "pd_time": pd_time_day,
        "pd_currency": pd_curr_day,
        "pd_news": pd_news_day,
        "pd_impact": pd_impact_day,
        "pd_forecast": pd_forecast_day,
        "pd_previous": pd_previous_day})
        df_hfxevents_list.append(df_hfxevents_day)
    count = count - 1
df_hfxevents_cat = pd.concat(df_hfxevents_list)
df_hfxevents_cat.to_csv('fxevents_hist.csv')
count=0
df_hfxevents_cat.head(20)

Forecast Date: 2017-01-22
Forcast Week String: 03
Current Year String: 2017
Current Year: 2017
https://www.babypips.com/economic-calendar?timezone=America%2FChicago&week=2017-W03
Forecast Date: 2017-01-29
Forcast Week String: 04
Current Year String: 2017
Current Year: 2017
https://www.babypips.com/economic-calendar?timezone=America%2FChicago&week=2017-W04
Forecast Date: 2017-02-05
Forcast Week String: 05
Current Year String: 2017
Current Year: 2017
https://www.babypips.com/economic-calendar?timezone=America%2FChicago&week=2017-W05
Forecast Date: 2017-02-12
Forcast Week String: 06
Current Year String: 2017
Current Year: 2017
https://www.babypips.com/economic-calendar?timezone=America%2FChicago&week=2017-W06
Forecast Date: 2017-02-19
Forcast Week String: 07
Current Year String: 2017
Current Year: 2017
https://www.babypips.com/economic-calendar?timezone=America%2FChicago&week=2017-W07
Forecast Date: 2017-02-26
Forcast Week String: 08
Current Year String: 2017
Current Year: 2017
https://ww

,pd_year,pd_month,pd_date,pd_weekday,pd_time,pd_currency,pd_news,pd_impact,pd_forecast,pd_previous
0,2017,Jan,18,Wednesday,01:00,EUR,German HICP(Final) (m/m),low,1.0%,1.0%
1,2017,Jan,18,Wednesday,01:00,EUR,German CPI (Final) (m/m),med,0.7%,0.7%
2,2017,Jan,18,Wednesday,01:00,EUR,German CPI (Final) (y/y),med,1.7%,1.7%
3,2017,Jan,18,Wednesday,01:00,EUR,German HICP (Final) (y/y),low,1.7%,1.7%
4,2017,Jan,18,Wednesday,03:30,GBP,Unemployment Rate,med,4.8%,4.8%
5,2017,Jan,18,Wednesday,03:30,GBP,Claimant Count Change Rate,med,2.3%,2.3%
6,2017,Jan,18,Wednesday,03:30,GBP,Average Earnings Index (3m y/y),med,2.6%,2.6%
7,2017,Jan,18,Wednesday,03:30,GBP,Claimant Count Change,med,5.0k,1.3k
8,2017,Jan,18,Wednesday,04:00,EUR,Core CPI (y/y),med,0.9%,0.9%
9,2017,Jan,18,Wednesday,04:00,EUR,CPI (y/y),high,1.1%,1.1%


In [29]:
file_path = Path("fxevents_hist.csv")
hist_data = pd.read_csv(file_path)
hist_data.head()

,Unnamed: 0,pd_year,pd_month,pd_date,pd_weekday,pd_time,pd_currency,pd_news,pd_impact,pd_forecast,pd_previous
0,0,2017,Jan,18,Wednesday,01:00,EUR,German HICP(Final) (m/m),low,1.0%,1.0%
1,1,2017,Jan,18,Wednesday,01:00,EUR,German CPI (Final) (m/m),med,0.7%,0.7%
2,2,2017,Jan,18,Wednesday,01:00,EUR,German CPI (Final) (y/y),med,1.7%,1.7%
3,3,2017,Jan,18,Wednesday,01:00,EUR,German HICP (Final) (y/y),low,1.7%,1.7%
4,4,2017,Jan,18,Wednesday,03:30,GBP,Unemployment Rate,med,4.8%,4.8%


In [30]:
hist_data['Date'] = pd.to_datetime(hist_data[['pd_month', 'pd_date', 'pd_year']].astype(str).agg('-'.join, axis=1))
hist_data.set_index(hist_data['Date'], inplace=True)
hist_data = hist_data.drop(columns=["pd_month", "pd_date", "pd_year", "pd_weekday", "Date"])
hist_data = hist_data.loc[:, ~hist_data.columns.str.contains('^Unnamed')]
hist_data.head(20)

,pd_time,pd_currency,pd_news,pd_impact,pd_forecast,pd_previous
Date,,,,,,
2017-01-18,01:00,EUR,German HICP(Final) (m/m),low,1.0%,1.0%
2017-01-18,01:00,EUR,German CPI (Final) (m/m),med,0.7%,0.7%
2017-01-18,01:00,EUR,German CPI (Final) (y/y),med,1.7%,1.7%
2017-01-18,01:00,EUR,German HICP (Final) (y/y),low,1.7%,1.7%
2017-01-18,03:30,GBP,Unemployment Rate,med,4.8%,4.8%
2017-01-18,03:30,GBP,Claimant Count Change Rate,med,2.3%,2.3%
2017-01-18,03:30,GBP,Average Earnings Index (3m y/y),med,2.6%,2.6%
2017-01-18,03:30,GBP,Claimant Count Change,med,5.0k,1.3k
2017-01-18,04:00,EUR,Core CPI (y/y),med,0.9%,0.9%


In [31]:
hist_data = hist_data[hist_data['pd_currency'].isin(['USD', 'EUR', 'GBP', 'JPY'])]
hist_data = hist_data[hist_data['pd_impact'].isin(['high'])]
hist_data.head(20)

,pd_time,pd_currency,pd_news,pd_impact,pd_forecast,pd_previous
Date,,,,,,
2017-01-18,04:00,EUR,CPI (y/y),high,1.1%,1.1%
2017-01-18,07:30,USD,CPI (m/m),high,0.3%,0.2%
2017-01-18,07:30,USD,Core CPI (m/m),high,0.2%,0.2%
2017-01-18,13:00,USD,Fed's Beige Book Released,high,NaN,NaN
2017-01-19,06:45,EUR,ECB - Deposit Rate,high,-0.4%,-0.4%
2017-01-19,06:45,EUR,ECB Interest Rate Decision,high,0.0%,0.0%
2017-01-19,07:30,EUR,ECB President Draghi gives press conference fo...,high,NaN,NaN
2017-01-20,03:30,GBP,Retail Sales (m/m),high,-0.1%,-0.1%
2017-01-20,03:30,GBP,Core Retail Sales (m/m),high,-0.4%,0.2%


In [32]:
hist_data.isnull().mean() * 100

pd_time         0.000000
pd_currency     0.000000
pd_news         0.000000
pd_impact       0.000000
pd_forecast    19.284188
pd_previous    15.705128
dtype: float64

In [33]:
hist_data["pd_forecast"] = hist_data["pd_forecast"].fillna("Unknown")
hist_data["pd_previous"] = hist_data["pd_previous"].fillna("Unknown")
hist_data.head(20)

,pd_time,pd_currency,pd_news,pd_impact,pd_forecast,pd_previous
Date,,,,,,
2017-01-18,04:00,EUR,CPI (y/y),high,1.1%,1.1%
2017-01-18,07:30,USD,CPI (m/m),high,0.3%,0.2%
2017-01-18,07:30,USD,Core CPI (m/m),high,0.2%,0.2%
2017-01-18,13:00,USD,Fed's Beige Book Released,high,Unknown,Unknown
2017-01-19,06:45,EUR,ECB - Deposit Rate,high,-0.4%,-0.4%
2017-01-19,06:45,EUR,ECB Interest Rate Decision,high,0.0%,0.0%
2017-01-19,07:30,EUR,ECB President Draghi gives press conference fo...,high,Unknown,Unknown
2017-01-20,03:30,GBP,Retail Sales (m/m),high,-0.1%,-0.1%
2017-01-20,03:30,GBP,Core Retail Sales (m/m),high,-0.4%,0.2%


In [34]:
hist_data.isnull().mean() * 100

pd_time        0.0
pd_currency    0.0
pd_news        0.0
pd_impact      0.0
pd_forecast    0.0
pd_previous    0.0
dtype: float64

In [35]:
hist_data = hist_data.rename(columns={
    "pd_time": "Time",
    "pd_currency": "Currency",
    "pd_news": "News", 
    "pd_impact": "Impact",
    "pd_forecast": "Forecast",
    "pd_previous": "Previous"})
hist_data.to_csv("Data/fxevents_hist_GSd.csv")
hist_data.head(20)

,Time,Currency,News,Impact,Forecast,Previous
Date,,,,,,
2017-01-18,04:00,EUR,CPI (y/y),high,1.1%,1.1%
2017-01-18,07:30,USD,CPI (m/m),high,0.3%,0.2%
2017-01-18,07:30,USD,Core CPI (m/m),high,0.2%,0.2%
2017-01-18,13:00,USD,Fed's Beige Book Released,high,Unknown,Unknown
2017-01-19,06:45,EUR,ECB - Deposit Rate,high,-0.4%,-0.4%
2017-01-19,06:45,EUR,ECB Interest Rate Decision,high,0.0%,0.0%
2017-01-19,07:30,EUR,ECB President Draghi gives press conference fo...,high,Unknown,Unknown
2017-01-20,03:30,GBP,Retail Sales (m/m),high,-0.1%,-0.1%
2017-01-20,03:30,GBP,Core Retail Sales (m/m),high,-0.4%,0.2%


In [36]:
historic_path = Path("Data/fxevents_hist_master.csv")
historicmstr_data = pd.read_csv(historic_path, index_col="Date", infer_datetime_format=True, parse_dates=True)
historicmstr_data.head(20)

,Time,Currency,News,Impact,Forecast,Previous,Direction
Date,,,,,,,
2017-01-18,4:00,EUR,CPI (y/y),high,1.10%,1.10%,1
2017-01-18,7:30,USD,CPI (m/m),high,0.30%,0.20%,1
2017-01-18,7:30,USD,Core CPI (m/m),high,0.20%,0.20%,1
2017-01-18,13:00,USD,Fed's Beige Book Released,high,Unknown,Unknown,0
2017-01-19,6:45,EUR,ECB - Deposit Rate,high,-0.40%,-0.40%,-1
2017-01-19,6:45,EUR,ECB Interest Rate Decision,high,0.00%,0.00%,-1
2017-01-19,7:30,EUR,ECB President Draghi gives press conference fo...,high,Unknown,Unknown,0
2017-01-20,3:30,GBP,Retail Sales (m/m),high,-0.10%,-0.10%,1
2017-01-20,3:30,GBP,Core Retail Sales (m/m),high,-0.40%,0.20%,1
